# Setting

This notebook solves the Biharmonic equation,

$$\nabla^4 u(x, y) = f(x, y)$$

on the unit square with source f given by

$$f(x, y) = 4 \pi^4 \sin(\pi x) \sin(\pi y)$$

and boundary conditions given by

$$u(x, y)         = 0$$
$$\nabla^2 u(x, y) = 0$$

using a discontinuous Galerkin formulation (interior penalty method).

# Implementation

In [ ]:
from dolfin import *
%matplotlib inline
import nbimporter
from boundary import apply_dirichlet_hermite
from dofs import list_hermite_dofs
import matplotlib.pyplot as pl
import numpy as np

# Optimization options for the form compiler
# MBD: Disable these until I've finished optimizedquadraturetransformer.py
#parameters["form_compiler"]["cpp_optimize"] = True
#parameters["form_compiler"]["optimize"] = True

# Make mesh ghosted for evaluation of DG terms
parameters["ghost_mode"] = "shared_facet"

def biharmonic(V):
    """
    This code is based on DOLFIN's homonymous demo, 
    (C) 2009 Kristian B. Oelgaard. Modified by Anders Logg, 2011
    
        V: FunctionSpace.
    """
    
    # Create mesh and define function space
    mesh = V.mesh()

    # Define Dirichlet boundary
    class DirichletBoundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary

    class Source(Expression):
        def eval(self, values, x):
            values[0] = 4.0*pi**4*sin(pi*x[0])*sin(pi*x[1])

    # Define boundary condition
    u0 = project(Constant(0.0), V)  # MBD: Need this for Hermite (interpolation doesn't work)
    bc = DirichletBC(V, u0, DirichletBoundary())

    # Define trial and test functions
    u = TrialFunction(V)
    v = TestFunction(V)

    # Define normal component, mesh size and right-hand side
    h = CellSize(mesh)
    h_avg = (h('+') + h('-'))/2.0
    n = FacetNormal(mesh)
    f = Source(degree=3)

    # Penalty parameter
    alpha = Constant(8.0)

    # Define bilinear form
    a = inner(div(grad(u)), div(grad(v)))*dx \
      - inner(avg(div(grad(u))), jump(grad(v), n))*dS \
      - inner(jump(grad(u), n), avg(div(grad(v))))*dS \
      + alpha/h_avg*inner(jump(grad(u),n), jump(grad(v),n))*dS

    # Define linear form
    L = f*v*dx
    
    # Solve variational problem
    u = Function(V)
    if V.ufl_element().family().lower() == 'hermite':
        A = assemble(a)
        b = assemble(L)
        apply_dirichlet_hermite(A, b, bc)
        solve(A, u.vector(), b)
    else:
        #%debug -b /home/fenics/local/lib/python2.7/site-packages/ffc/quadrature/quadraturetransformerbase.py:347 solve(a == L, u, bc)
        solve(a == L, u, bc)
    
    return u

# Results

In [ ]:
W = FunctionSpace(UnitSquareMesh(15, 15, 'crossed'), "Lagrange", 3)
good = biharmonic(W)

In [ ]:
_ = plot(good, title="Lagrange elements", cmap='hot')

In [ ]:
V = FunctionSpace(W.mesh(), "Hermite", 3)
bad = biharmonic(V)

In [ ]:
_ = plot(bad, title="Hermite elements", cmap='hot')

We compute now the pointwise relative error, defined as the quotient $\frac{u_h - u_l}{1+u_l}$:

In [ ]:
pl.figure(figsize=(12,6))
xx = np.arange(0, 1, 0.01)
for y in np.linspace(0.1,0.9,4):
    pl.plot(xx, [np.abs((z(x,y)))/(1+np.abs(good(x,y))) for x in xx],
            label='diff@%.1f' % y)
pl.title("Relative error at several ordinates")
_ = pl.legend()

The simple difference is strange:

In [ ]:
xx = np.linspace(0.,1,100)
pl.figure(figsize=(12,12))
for y in np.linspace(0.3,0.6,3):
    pl.plot(xx, [bad(x, y) - good(x, y) for x in xx], label='@%.1f' % y)
pl.title("$u_h - u_l$")
_ = pl.legend()

The maximal difference is indeed around 0.4%:

In [ ]:
gv = good.vector().array()
bv = bad.vector().array()
lagrange_dofs = list(set(range(V.dim())) - set(list_hermite_dofs(V)))
(gv.min(), gv.max(), bv[lagrange_dofs].min(), bv[lagrange_dofs].max())

The following plot is weird:

In [ ]:
z = project(good-bad, V)
print(norm(z))
_ = plot(z, title="Difference", cmap='bone')

Save the solutions to files:

In [ ]:
File("biharmonic-hermite.pvd") << bad
File("biharmonic-lagrange.pvd") << good